# Creation of test dataset for Multi-center tetrahedral prediction

## Import section

In [1]:
import os

import pandas as pd
from tqdm import tqdm
from multiprocess.pool import Pool

## Load initial datset

Which dataset to choose: tox, has most such elements. (could also fuze datasets but should be enough for miniature dataset, merging datasets create other problems)

In [2]:
df = pd.read_csv("hyperoptimization/src/tox21/raw/tox21.csv.gz")

In [3]:
df.head()

,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53,mol_id,smiles
0,0.0,0.0,1.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,TOX3021,CCOc1ccc2nc(S(N)(=O)=O)sc2c1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,TOX3020,CCN1C(=O)NC(c2ccccc2)C1=O
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,TOX3024,CC[C@]1(O)CC[C@H]2[C@@H]3CCC4=CCCC[C@@H]4[C@H]...
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,TOX3027,CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX20800,CC(O)(P(=O)(O)O)P(=O)(O)O


## Filter out less than 2 stereo center molecules

In [4]:
def filter_mc_tetra_mol(
        df_entry,
        smiles_name: str = "smiles"
):
    """
    Returns true if molecule has multiple chiral centers.

    :param df_entry: pandas row object
    :param smiles_name: name of the columns under which the stereochemical SMILES can be queried from the
        df_entry
    :return: whether or not the molecule has multiple chiral centers (plus the idx and the entry before that)
    """
    from ptgnn.features.chienn.molecule3d import smiles_to_3d_mol
    from rdkit.Chem import AllChem

    # unpack passed object
    idx, entry = df_entry

    # extract smiles string
    smiles = entry[smiles_name]

    # get molecule
    molecule = smiles_to_3d_mol(
        smiles=smiles,
        max_number_of_atoms=100,
        max_number_of_attempts=100
    )

    # if molecule cannot be rendered (i.e. is None) return appropriate information
    if molecule is None:
        return idx, df_entry, False

    # does molecule have multiple marked chiral centers
    return idx, entry, len(AllChem.FindMolChiralCenters(molecule)) == 2

In [5]:
with Pool(processes=os.cpu_count()) as p:
    df_collection = list(p.map(
        filter_mc_tetra_mol,
        tqdm(df.iterrows(), total=len(df))
    ))

100%|██████████| 7831/7831 [00:26<00:00, 293.06it/s] 


In [6]:
df_collection = pd.DataFrame([
    elem[1]
    for elem in df_collection
    if elem[2] == 1
]).reset_index(drop='index')

## Produce all possible combinations of @ or @@

In [7]:
def worker_create_all_combinations(
        df_entry,
        smiles_name: str = 'smiles'
):
    # unpack passed object
    idx, entry = df_entry

    # extract smiles string
    smiles = entry[smiles_name]

    # replace all double @ with normal @... wait but different forms? urgh the smiles may be different...
    # what in the first place is the label?
    # if I make it such that EXACTLY 2, then u/l prediction is possible. or distribution of max(|R|, |S|) in general

    # get clearned smiles version with only one @
    cleaned_smiles = smiles.replace("@@", "@")

    # split it
    split_smiles = cleaned_smiles.split("@")
    if len(split_smiles) != 3:
        print(f"SMILES {smiles} dropped as too many @ in smiles")
        return pd.DataFrame()

    # it is basically guaranteed that this results in 3 parts as there must be 2 centers in there
    return pd.DataFrame({
        "smiles": [
            split_smiles[0] + "@" + split_smiles[1] + "@" + split_smiles[2],
            split_smiles[0] + "@" + split_smiles[1] + "@@" + split_smiles[2],
            split_smiles[0] + "@@" + split_smiles[1] + "@" + split_smiles[2],
            split_smiles[0] + "@@" + split_smiles[1] + "@@" + split_smiles[2],
        ],
    })

In [8]:
df_collection

,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53,mol_id,smiles
0,0.0,0.0,1.0,NaN,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,TOX22631,CN[C@@H]1C[C@@H](c2ccc(Cl)c(Cl)c2)c2ccccc21
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,TOX25538,CN(C)CCn1nnnc1SCC1=C(C(=O)O)N2C(=O)[C@@H](NC(=...
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX25530,CCN[C@H]1C[C@H](C)S(=O)(=O)c2sc(S(N)(=O)=O)cc21
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,TOX28633,CCCCCCCCCCCCCCCC(=O)O[C@@H]1CC(C)=C(/C=C/C(C)=...
4,0.0,0.0,1.0,NaN,1.0,0.0,0.0,0.0,1.0,0.0,NaN,0.0,TOX27264,Oc1ccc2c(c1)OC[C@@H](N1CCC(O)(c3ccc(F)cc3)CC1)...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,1.0,0.0,TOX28548,CCC[C@@]1(CCc2ccccc2)CC(O)=C([C@H](CC)c2cccc(N...
223,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX25342,CC(=O)OCC1=C(C(=O)O)N2C(=O)[C@@H](N)[C@H]2SC1
224,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX4721,CC1(C)[C@@H]2CC[C@@]1(C)C(=O)C2
225,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,TOX27852,OC[C@H](Cc1cccc(O)c1)[C@H](CO)Cc1cccc(O)c1


In [9]:
df_collection = pd.concat(
    list(map(
        worker_create_all_combinations,
        tqdm(df_collection.iterrows(), total=len(df_collection))
    ))
).reset_index(drop='index')

100%|██████████| 227/227 [00:00<00:00, 16215.18it/s]

SMILES CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(CSc3nnnn3C)CS[C@H]12)c1csc(N)n1.CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(CSc3nnnn3C)CS[C@H]12)c1csc(N)n1 dropped as too many @ in smiles
SMILES COc1ccc(C[C@@H](C)NC[C@H](O)c2ccc(O)c(NC=O)c2)cc1.COc1ccc(C[C@@H](C)NC[C@H](O)c2ccc(O)c(NC=O)c2)cc1 dropped as too many @ in smiles
SMILES O=C1O[C@H]([C@@H](O)CO)C([O-])=C1O.O=C1O[C@H]([C@@H](O)CO)C([O-])=C1O dropped as too many @ in smiles
SMILES C[C@@H]1CN([C@H]2CC[C@](C#N)(c3ccc(F)cc3)CC2)CC[C@]1(C(=O)O)c1ccccc1 dropped as too many @ in smiles
SMILES CC(C)c1c(C(=O)Nc2ccccc2)c(-c2ccccc2)c(-c2ccc(F)cc2)n1CC[C@@H](O)C[C@@H](O)CC(=O)[O-].CC(C)c1c(C(=O)Nc2ccccc2)c(-c2ccccc2)c(-c2ccc(F)cc2)n1CC[C@@H](O)C[C@@H](O)CC(=O)[O-] dropped as too many @ in smiles
SMILES COc1ccc(C[C@H](C)NC[C@@H](O)c2ccc(O)c(NC=O)c2)cc1.COc1ccc(C[C@H](C)NC[C@@H](O)c2ccc(O)c(NC=O)c2)cc1 dropped as too many @ in smiles


In [10]:
df_collection

,smiles
0,CN[C@H]1C[C@H](c2ccc(Cl)c(Cl)c2)c2ccccc21
1,CN[C@H]1C[C@@H](c2ccc(Cl)c(Cl)c2)c2ccccc21
2,CN[C@@H]1C[C@H](c2ccc(Cl)c(Cl)c2)c2ccccc21
3,CN[C@@H]1C[C@@H](c2ccc(Cl)c(Cl)c2)c2ccccc21
4,CN(C)CCn1nnnc1SCC1=C(C(=O)O)N2C(=O)[C@H](NC(=O...
...,...
879,OC[C@@H](Cc1cccc(O)c1)[C@@H](CO)Cc1cccc(O)c1
880,O=C(O[C@H]1Cc2c(O)cc(O)cc2O[C@H]1c1cc(O)c(O)c(...
881,O=C(O[C@H]1Cc2c(O)cc(O)cc2O[C@@H]1c1cc(O)c(O)c...
882,O=C(O[C@@H]1Cc2c(O)cc(O)cc2O[C@H]1c1cc(O)c(O)c...


In [11]:
df_collection.drop_duplicates(subset=['smiles'], inplace=True)

In [12]:
df_collection

,smiles
0,CN[C@H]1C[C@H](c2ccc(Cl)c(Cl)c2)c2ccccc21
1,CN[C@H]1C[C@@H](c2ccc(Cl)c(Cl)c2)c2ccccc21
2,CN[C@@H]1C[C@H](c2ccc(Cl)c(Cl)c2)c2ccccc21
3,CN[C@@H]1C[C@@H](c2ccc(Cl)c(Cl)c2)c2ccccc21
4,CN(C)CCn1nnnc1SCC1=C(C(=O)O)N2C(=O)[C@H](NC(=O...
...,...
879,OC[C@@H](Cc1cccc(O)c1)[C@@H](CO)Cc1cccc(O)c1
880,O=C(O[C@H]1Cc2c(O)cc(O)cc2O[C@H]1c1cc(O)c(O)c(...
881,O=C(O[C@H]1Cc2c(O)cc(O)cc2O[C@@H]1c1cc(O)c(O)c...
882,O=C(O[C@@H]1Cc2c(O)cc(O)cc2O[C@H]1c1cc(O)c(O)c...


In [13]:
def assign_lu_label(df_entry):
    # unpack passed object
    idx, entry = df_entry

    # extract smiles string
    smiles = entry['smiles']

    # render molecule
    from ptgnn.features.chienn.molecule3d import smiles_to_3d_mol
    molecule = smiles_to_3d_mol(smiles, max_number_of_atoms=100, max_number_of_attempts=100)

    if molecule is None:
        return None

    # create the label, thus first fetch stereo centers
    from rdkit import Chem
    center_one, center_two = Chem.FindMolChiralCenters(molecule)

    binary_label = center_one[1] == center_two[1]

    entry['MC_label'] = "L" if binary_label else "U"
    entry['MC_label_binary'] = 1 if binary_label else 0

    return entry

In [14]:
with Pool(processes=os.cpu_count()) as p:
    df_collection = list(p.map(
        assign_lu_label,
        tqdm(df_collection.iterrows(), total=len(df_collection))
    ))

100%|██████████| 852/852 [00:09<00:00, 92.68it/s]  


In [15]:
df_collection

[smiles             CN[C@H]1C[C@H](c2ccc(Cl)c(Cl)c2)c2ccccc21
 MC_label                                                   U
 MC_label_binary                                        False
 Name: 0, dtype: object,
 smiles             CN[C@H]1C[C@@H](c2ccc(Cl)c(Cl)c2)c2ccccc21
 MC_label                                                    L
 MC_label_binary                                          True
 Name: 1, dtype: object,
 smiles             CN[C@@H]1C[C@H](c2ccc(Cl)c(Cl)c2)c2ccccc21
 MC_label                                                    L
 MC_label_binary                                          True
 Name: 2, dtype: object,
 smiles             CN[C@@H]1C[C@@H](c2ccc(Cl)c(Cl)c2)c2ccccc21
 MC_label                                                     U
 MC_label_binary                                          False
 Name: 3, dtype: object,
 smiles             CN(C)CCn1nnnc1SCC1=C(C(=O)O)N2C(=O)[C@H](NC(=O...
 MC_label                                                           U
 MC_

In [16]:
df_collection = pd.DataFrame([elem for elem in df_collection if elem is not None])
df_collection

,smiles,MC_label,MC_label_binary
0,CN[C@H]1C[C@H](c2ccc(Cl)c(Cl)c2)c2ccccc21,U,False
1,CN[C@H]1C[C@@H](c2ccc(Cl)c(Cl)c2)c2ccccc21,L,True
2,CN[C@@H]1C[C@H](c2ccc(Cl)c(Cl)c2)c2ccccc21,L,True
3,CN[C@@H]1C[C@@H](c2ccc(Cl)c(Cl)c2)c2ccccc21,U,False
4,CN(C)CCn1nnnc1SCC1=C(C(=O)O)N2C(=O)[C@H](NC(=O...,U,False
...,...,...,...
879,OC[C@@H](Cc1cccc(O)c1)[C@@H](CO)Cc1cccc(O)c1,L,True
880,O=C(O[C@H]1Cc2c(O)cc(O)cc2O[C@H]1c1cc(O)c(O)c(...,L,True
881,O=C(O[C@H]1Cc2c(O)cc(O)cc2O[C@@H]1c1cc(O)c(O)c...,U,False
882,O=C(O[C@@H]1Cc2c(O)cc(O)cc2O[C@H]1c1cc(O)c(O)c...,U,False


In [1]:
from ptgnn.dataset.mc_dataset import MCDataset

In [2]:
train_mc = MCDataset(**{'transformation_parameters': {'tetrahedral_chiral': False, 'multi_stereo_center_dia': True, 'multi_stereo_center_dia_mode': 1, 'k': 3}, 'type': 'mc', 'mask_chiral_tags': True, 'graph_mode': 'edge', 'transformation_mode': 'permutation_tree', 'root': 'D:\\DATEN\\Masterarbeit_PTGNN\\notebooks\\hyperoptimization\\src\\mc'})
train_mc

MCDataset(578)

In [3]:
val_mc = MCDataset(split="val", **{'transformation_parameters': {'tetrahedral_chiral': False, 'multi_stereo_center_dia': True, 'multi_stereo_center_dia_mode': 1, 'k': 3}, 'type': 'mc', 'mask_chiral_tags': True, 'graph_mode': 'edge', 'transformation_mode': 'permutation_tree', 'root': 'D:\\DATEN\\Masterarbeit_PTGNN\\notebooks\\hyperoptimization\\src\\mc'})
val_mc

MCDataset(88)

In [4]:
test_mc = MCDataset(split="test", **{'transformation_parameters': {'tetrahedral_chiral': False, 'multi_stereo_center_dia': True, 'multi_stereo_center_dia_mode': 1, 'k': 3}, 'type': 'mc', 'mask_chiral_tags': True, 'graph_mode': 'edge', 'transformation_mode': 'permutation_tree', 'root': 'D:\\DATEN\\Masterarbeit_PTGNN\\notebooks\\hyperoptimization\\src\\mc'})
test_mc

MCDataset(167)

In [9]:
counter = 0
for elem in train_mc:
    display(elem, elem.y, elem.ptree)

    counter += 1
    if counter > 10:
        break

Data(x=[72, 118], edge_index=[2, 198], edge_attr=[198, 80], pos=[72, 6], parallel_node_index=[72], circle_index=[72], ptree=[72], initial_map=[288], layer0_order_matrix=[1], layer0_type_mask=[1], layer0_pooling=[1], num_layer=[1], layer1_order_matrix=[1], layer1_type_mask=[1], layer1_pooling=[1], layer2_order_matrix=[1], layer2_type_mask=[1], layer2_pooling=[1], layer3_order_matrix=[1], layer3_type_mask=[1], layer3_pooling=[1], y=[1])

tensor([0])

['{"S": [0, 1, {"P": [3, 7, 5]}]}',
 '{"S": [1, 0, {"P": [9, 11]}]}',
 '{"S": [2, 3, {"P": [1, 5, 7]}]}',
 '{"S": [3, 2]}',
 '{"S": [4, 5, {"P": [1, 7, 3]}]}',
 '{"S": [5, 4]}',
 '{"S": [6, 7, {"P": [1, 3, 5]}]}',
 '{"S": [7, 6]}',
 '{"S": [8, 9, {"P": [0, 11]}]}',
 '{"S": [9, 8, {"P": [13, 15, 17]}]}',
 '{"S": [10, 11, {"P": [0, 9]}]}',
 '{"S": [11, 10]}',
 '{"S": [12, 13, {"P": [{"P": [{"Z": [25, 29, 27]}, {"Z": [8, 17, 15]}]}, {"P": [{"Z": [27, 29, 25]}, {"Z": [15, 17, 8]}]}]}]}',
 '{"S": [13, 12, {"P": [19, 23, 21]}]}',
 '{"S": [14, 15, {"P": [8, 13, 17]}]}',
 '{"S": [15, 14, {"P": [54, 68]}]}',
 '{"S": [16, 17, {"P": [8, 15, 13]}]}',
 '{"S": [17, 16]}',
 '{"S": [18, 19, {"P": [12, 21, 23]}]}',
 '{"S": [19, 18, {"P": [{"P": [{"Z": [25, 29, 27]}, {"Z": [8, 17, 15]}]}, {"P": [{"Z": [27, 29, 25]}, {"Z": [15, 17, 8]}]}]}]}',
 '{"S": [20, 21, {"P": [12, 23, 19]}]}',
 '{"S": [21, 20]}',
 '{"S": [22, 23, {"P": [12, 19, 21]}]}',
 '{"S": [23, 22]}',
 '{"S": [24, 25, {"P": [18, 27, 29]}]}',


Data(x=[72, 118], edge_index=[2, 198], edge_attr=[198, 80], pos=[72, 6], parallel_node_index=[72], circle_index=[72], ptree=[72], initial_map=[288], layer0_order_matrix=[1], layer0_type_mask=[1], layer0_pooling=[1], num_layer=[1], layer1_order_matrix=[1], layer1_type_mask=[1], layer1_pooling=[1], layer2_order_matrix=[1], layer2_type_mask=[1], layer2_pooling=[1], layer3_order_matrix=[1], layer3_type_mask=[1], layer3_pooling=[1], y=[1])

tensor([1])

...

tensor([0])

['{"S": [0, 1, {"P": [3, 5, 7]}]}',
 '{"S": [1, 0, {"P": [9, 11, 13]}]}',
 '{"S": [2, 3, {"P": [1, 7, 5]}]}',
 '{"S": [3, 2]}',
 '{"S": [4, 5, {"P": [1, 3, 7]}]}',
 '{"S": [5, 4]}',
 '{"S": [6, 7, {"P": [1, 5, 3]}]}',
 '{"S": [7, 6]}',
 '{"S": [8, 9, {"P": [0, 13, 11]}]}',
 '{"S": [9, 8, {"P": [15, 17, 19]}]}',
 '{"S": [10, 11, {"P": [0, 9, 13]}]}',
 '{"S": [11, 10]}',
 '{"S": [12, 13, {"P": [0, 11, 9]}]}',
 '{"S": [13, 12]}',
 '{"S": [14, 15, {"P": [8, 19, 17]}]}',
 '{"S": [15, 14, {"P": [21, 23, 25]}]}',
 '{"S": [16, 17, {"P": [8, 15, 19]}]}',
 '{"S": [17, 16]}',
 '{"S": [18, 19, {"P": [8, 17, 15]}]}',
 '{"S": [19, 18]}',
 '{"S": [20, 21, {"P": [14, 25, 23]}]}',
 '{"S": [21, 20, {"P": [27, 31, 29]}]}',
 '{"S": [22, 23, {"P": [14, 21, 25]}]}',
 '{"S": [23, 22]}',
 '{"S": [24, 25, {"P": [14, 23, 21]}]}',
 '{"S": [25, 24]}',
 '{"S": [26, 27, {"P": [20, 29, 31]}]}',
 '{"S": [27, 26, {"P": [33, 37, 35]}]}',
 '{"S": [28, 29, {"P": [20, 31, 27]}]}',
 '{"S": [29, 28]}',
 '{"S": [30, 31, {"P"

Data(x=[386, 118], edge_index=[2, 1110], edge_attr=[1110, 80], pos=[386, 6], parallel_node_index=[386], circle_index=[386], ptree=[386], initial_map=[1514], layer0_order_matrix=[1], layer0_type_mask=[1], layer0_pooling=[1], num_layer=[1], layer1_order_matrix=[1], layer1_type_mask=[1], layer1_pooling=[1], layer2_order_matrix=[1], layer2_type_mask=[1], layer2_pooling=[1], layer3_order_matrix=[1], layer3_type_mask=[1], layer3_pooling=[1], y=[1])

tensor([0])

['{"S": [0, 1, {"P": [3, 5, 7]}]}',
 '{"S": [1, 0, {"P": [9, 11, 13]}]}',
 '{"S": [2, 3, {"P": [1, 7, 5]}]}',
 '{"S": [3, 2]}',
 '{"S": [4, 5, {"P": [1, 3, 7]}]}',
 '{"S": [5, 4]}',
 '{"S": [6, 7, {"P": [1, 5, 3]}]}',
 '{"S": [7, 6]}',
 '{"S": [8, 9, {"P": [0, 13, 11]}]}',
 '{"S": [9, 8, {"P": [15, 17, 19]}]}',
 '{"S": [10, 11, {"P": [0, 9, 13]}]}',
 '{"S": [11, 10]}',
 '{"S": [12, 13, {"P": [0, 11, 9]}]}',
 '{"S": [13, 12]}',
 '{"S": [14, 15, {"P": [8, 19, 17]}]}',
 '{"S": [15, 14, {"P": [21, 23, 25]}]}',
 '{"S": [16, 17, {"P": [8, 15, 19]}]}',
 '{"S": [17, 16]}',
 '{"S": [18, 19, {"P": [8, 17, 15]}]}',
 '{"S": [19, 18]}',
 '{"S": [20, 21, {"P": [14, 25, 23]}]}',
 '{"S": [21, 20, {"P": [27, 31, 29]}]}',
 '{"S": [22, 23, {"P": [14, 21, 25]}]}',
 '{"S": [23, 22]}',
 '{"S": [24, 25, {"P": [14, 23, 21]}]}',
 '{"S": [25, 24]}',
 '{"S": [26, 27, {"P": [20, 29, 31]}]}',
 '{"S": [27, 26, {"P": [33, 37, 35]}]}',
 '{"S": [28, 29, {"P": [20, 31, 27]}]}',
 '{"S": [29, 28]}',
 '{"S": [30, 31, {"P"